In [1]:
import cantera as ct
import os
import shutil
import pandas as pd
import numpy as np

In [2]:
or_p = '/work/westgroup/chao/bm_project/base_original/binding_energies/52.0_c-6.75o-4.75/cantera.yaml'
bm_p = '/work/westgroup/chao/bm_project/base_bm/binding_energies/52.0_c-6.75o-4.75/cantera.yaml'
gas_or = ct.Solution(or_p, 'gas')
surf_or = ct.Interface(or_p, 'surface1', [gas_or])
gas_or.species()


/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in cp/R detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  10.615319744584205
	Value computed using high-temperature polynomial: 10.773680323194302

  after removing the cwd from sys.path.
/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in h/RT detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  -23.034356052440977
	Value computed using high-temperature polynomial: -23.046264132668917

  after removing the cwd from sys.path.
/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: NasaPoly2::validate: 
For species CH4OX(43), discontinuity in cp/R detected at Tmid = 850.41
	Value computed using low-temperature polynomi

[<Species Ar>,
 <Species Ne>,
 <Species N2>,
 <Species CH4(2)>,
 <Species O2(3)>,
 <Species CO2(4)>,
 <Species H2O(5)>,
 <Species H2(6)>,
 <Species CO(7)>,
 <Species C2H6(8)>,
 <Species CH2O(9)>,
 <Species CH3(10)>,
 <Species C3H8(11)>,
 <Species H(12)>,
 <Species C2H5(13)>,
 <Species CH3OH(14)>,
 <Species HCO(15)>,
 <Species CH3CHO(16)>,
 <Species OH(17)>,
 <Species C2H4(18)>,
 <Species CH3OO(20)>,
 <Species C2H4(62)>]

In [3]:
benchmark_p = '/work/westgroup/chao/bm_project/base_bm/binding_energies/52.0_c-6.75o-4.75/gas_profiles/gas_out0.6.csv'
df_benchmark = pd.read_csv(benchmark_p)
ch4_benchmark_1045 = df_benchmark.iloc[3, 1:][1045]
diffs = []
for i in range(surf_or.n_reactions):
    p_target = f'bm2or/rxn{i}/gas_profiles/gas_out0.6.csv'
    df_target = pd.read_csv(p_target)
    ch4_bm_1045 = df_target.iloc[3, 1:][1045]
    diff = abs((ch4_bm_1045 - ch4_benchmark_1045) / ch4_benchmark_1045)
    diffs.append(diff)
    if diff > 0.5:
        print(i)
        print(surf_or.reaction(i))

In [4]:
top_10_diffs = np.argsort(diffs)[-10:]
for i in top_10_diffs:
    print(diffs[i])
    print(i)
    print(surf_or.reaction(i))

0.0007256933275590538
42
CH4X(24) + X(1) <=> CH3X(27) + HX(21)
0.0007749285431643894
5
COX(23) + OX(25) <=> CO2X(22) + X(1)
0.001640309864500841
9
CHX(28) + HX(21) <=> CH2X(26) + X(1)
0.002931327661451421
17
HX(21) + OX(25) <=> OHX(31) + X(1)
0.005475670209006894
56
CH4X(24) + OX(25) <=> CH3X(27) + OHX(31)
0.012069864398532876
15
CH4(2) + OX(25) + X(1) <=> CH3X(27) + OHX(31)
0.012102289755625077
34
CO2(4) + 2 X(1) <=> COX(23) + OX(25)
0.013493164035424911
16
CO2X(22) + HX(21) <=> COX(23) + OHX(31)
0.040962175134880735
32
CH4(2) + X(1) <=> CH4X(24)
0.251848387315748
35
H2O(5) + 2 X(1) <=> HX(21) + OHX(31)


In [5]:
gas_bm = ct.Solution(bm_p, 'gas')
surf_bm = ct.Interface(bm_p, 'surface1', [gas_bm])

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in cp/R detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  10.615319744584205
	Value computed using high-temperature polynomial: 10.773680323194302

  
/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in h/RT detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  -23.034356052440977
	Value computed using high-temperature polynomial: -23.046264132668917

  
/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: NasaPoly2::validate: 
For species CH4OX(43), discontinuity in cp/R detected at Tmid = 850.41
	Value computed using low-temperature polynomial:  11.305535344771027
	Value computed using high-temperature polynomial:

In [6]:
print(format(surf_or.forward_rate_constants[35],'.0e'), format(surf_bm.forward_rate_constants[35], '.0e'))

1e+05 4e+10


In [7]:
# check the rate of progress
or_rp = '/work/westgroup/chao/bm_project/base_original/binding_energies/52.0_c-6.75o-4.75/rates_of_progress/net_rates_0.6.csv'
bm_rp = '/work/westgroup/chao/bm_project/base_bm/binding_energies/52.0_c-6.75o-4.75/rates_of_progress/net_rates_0.6.csv'
df_or_rp = pd.read_csv('/work/westgroup/chao/bm_project/base_original/binding_energies/52.0_c-6.75o-4.75/rates_of_progress/net_rates_0.6.csv')
df_bm_rp = pd.read_csv('/work/westgroup/chao/bm_project/base_bm/binding_energies/52.0_c-6.75o-4.75/rates_of_progress/net_rates_0.6.csv')

In [8]:
print(df_or_rp.iloc[1045,36], df_bm_rp.iloc[1045,36])

-5.190518717741798e-06 -0.0016066875648234732
